# Calibration curves toolbox #

## Import potřebných knihoven

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Import naměřených dat z OpenSoundMeteru

Vstupní parametry:
* `path` - cesta k CSV spuboru vyexportovaného z OSM

Návratové hodnoty:
* Pandas `DataFrame` - naimportovaná data :
    * frekvenční charakteristika - pod atributem `Magnitude`
    * fázová odezva - pod atributem `Phase`
    * koherence měření - pod atributem `Coherence`

In [2]:
def import_data_from_osm( path : str ) -> pd.DataFrame:
    return pd.read_csv( path, names=["Freq", "Magnitude", "Phase", "Coherence"] )

## Ořez naměřených dat do rozsahu slyšitelné části spektra

Vstupní parametry:
* `source_df` - Pandas `DataFrane` s naměřenými daty se strukturou popsanou [výše](#Import-naměřených-dat-z-OpenSoundMeteru)

Návratové hodnoty:
* Pandas `DataFrame` - `DataFrame` se stejnou strukturou (viz [výše](#Import-naměřených-dat-z-OpenSoundMeteru)) - ve spektru zůžen na interval $20 Hz$ až $20 kHz$

In [3]:
def cut_to_audible_spectrum( source_df : pd.DataFrame ) -> pd.DataFrame:
    cut_df = source_df.drop(source_df.loc[(source_df["Freq"] < 20) | (source_df["Freq"] > 20000)].index)
    return cut_df

## Generování rozsahů pro 1/n oktávový smoothing

Pokud pro export dat nepoužijeme Smaart a možnost exportu dat s 1/n oktávovým smoothingem, či OSM s FFT nastaveném v módu `LTW` (Logarithmic time window), pak se o průměrování musíme postarat sami.

Pro tuto potřebu musíme být schopni vygenerovat intervaly frekvencí jednotlivých "oktáv" pro na kterých budeme průměrovat.

Vstupní parametry:
* `noct` - pokud chceme 1/n oktávové průměrování (smoothing), tak tento parametr je právě ono `n` (tedy například pro 1/3-oktávové průměrování je `noct` = 3)

Návratová hodnota:
* Slovník (`Dictionary`) kde klíčem je vždy centrální frekvece dané oktávy a hodnotu k takovéme klíči je pak slovník s dvěma položkami:
    * `lo` - dolní ohraničující frekvence oktávy
    * `hi` - horní ohraničující frekvence oktávy

In [6]:
def generate_bin_data( noct : int ) -> dict:
    freq_dict = {}

    reference_freq = 1000
    lower_threshold = 20
    upper_threshold = 20000

    #Lower freqs:
    #print("Lower")
    freq = reference_freq
    lo = lower_threshold
    while ( lo >= lower_threshold):
        central_freq = freq
        lo = central_freq / 2**(1/(noct * 2))
        hi = central_freq * 2**(1/(noct * 2))
        #print("F: {0}, LO: {1}, HI: {2}".format(central_freq, lo, hi))
        freq = math.trunc(freq / 2**(1/noct))
        freq_dict[central_freq] = {}
        freq_dict[central_freq]["lo"] = lo
        freq_dict[central_freq]["hi"] = hi
    
    #Higher freqs
    #print("Higher")
    freq = math.trunc(reference_freq * 2**(1/noct))
    hi = upper_threshold
    while (hi <= upper_threshold):
        central_freq = freq
        lo = central_freq / 2**(1/(noct * 2))
        hi = central_freq * 2**(1/(noct * 2))
        #print("F: {0}, LO: {1}, HI: {2}".format(central_freq, lo, hi))
        freq = math.trunc(freq * 2**(1/noct), )
        freq_dict[central_freq] = {}
        freq_dict[central_freq]["lo"] = lo
        freq_dict[central_freq]["hi"] = hi

    return freq_dict

## Smoothing (průměrování)

Tato funkce provede samotné průměrování dat.

Vstupní parametry:
* `source_data` - `DatFrame`se strukturou popsanou v [výše](#Import-naměřených-dat-z-OpenSoundMeteru)
* `bands` - slovník (mapa) s průměrovacími intervaly (viz [výše](#Generování-rozsahů-pro-1/n-oktávový-smoothing))

Návratové hodnoty:
* `DataFrame` obsahující zprůměrovaná data frekvenční charakteristiky

Průměr se zde počítá jako aritmetický, tedy pro n bodů $m_1, m_2, \dots, m_n$ v dané oktávě:
$$
    \frac{\sum_{i = 1}^{n} m_i}{n}
$$

Do budoucna by si tato funkce zasloužila vylepšení o možnsot volby typu průměrování

In [7]:
def generate_oct_view( source_data : pd.DataFrame, bands : dict ) -> pd.DataFrame:
    value_dict = {}

    for key in bands.keys():
        lo = bands[key]["lo"]
        hi = bands[key]["hi"]

        values_df = source_data.loc[(source_data["Freq"] >= lo) & (source_data["Freq"] <= hi)]
        if (values_df.empty):
            avg = 0
        else:
            avg = values_df["Magnitude"].sum() / len(values_df["Magnitude"].index)
        
        #print(avg)

        value_dict[key] = avg
        
    return_df = pd.DataFrame.from_dict(value_dict, orient="index", columns=["Magnitude"])
    return_df.sort_index(inplace=True)

    return return_df

## Dvoupásmová kalibrace

Pokud kalibrujeme dvoupásmově, tedy zvlášť s basovkou a zvlášť pro středy a výšky, hodí se možnost zkombinovat zprůměrované křivky.

Vstupní parametry:
* `lo_trace` - kalibrační křivka pro spodní část spektra (`DataFrame` - již zprůměrovaná dle [předchozího bodu](#Smoothing-(průměrování)))
* `hi_trace` - kalibrační křivka pro horní část spektra (`DataFrame` - již zprůměrovaná dle [předchozího bodu](#Smoothing-(průměrování))))
* `join_freq` - frekvence (oktáva), na které se kalibrační křivky mají napojit

Návratové hodnoty:
* `DataFrame` obsahující spojenou kalibrační křivku (index je frekvence, atribut `Magnitude` pak obsahuje samotné hodnoty frekvenční charakteristiky

In [8]:
def combine_lo_hi_traces( lo_trace : pd.DataFrame, hi_trace : pd.DataFrame, join_freq : float ) -> pd.DataFrame:
    lo_trace_join_val = lo_trace.loc[lo_trace.index == join_freq].values[0]
    hi_trace_join_val = hi_trace.loc[hi_trace.index == join_freq].values[0]

    diff = hi_trace_join_val - lo_trace_join_val

    lo_trace_cut = lo_trace.loc[lo_trace.index < join_freq]

    for each_index in lo_trace_cut.index.values:
        prev_val = lo_trace.loc[lo_trace.index == each_index].values[0]
        new_val = prev_val + diff
        hi_trace.loc[hi_trace.index == each_index] = new_val
    
    return hi_trace

## Normalizace křivky

Tato funkce křivku znormalizuje - tedy posune ji tak, že na 1kHz bude 0 (0dB)

Vstupní parametry:
* `curve` - `DataFrame` obsahující již zprůměrovanou a případně sloučenou kalibrační křivku

Výstup:
* `DataFrame` obsahující finální kalibrační křivku

In [9]:
def normalize_curve( curve : pd.DataFrame ) -> pd.DataFrame:
    one_k_val = curve.loc[curve.index == 1000].values[0]
    curve["Magnitude"] -=one_k_val
    return curve

## Export křivky

Funkce sloužící pro export hotové křivk do textového souboru

Vstupní parametry:
* `curve` -`DataFrame` obsahující hotovou křivku
* `path` - cesta k výstupnímu souboru (soubor bude vytvořen)

In [10]:
def export_curve( curve : pd.DataFrame, path : str ) -> None:
    curve["Magnitude"] = curve["Magnitude"].round(decimals=2)
    curve.to_csv(path, header=False, sep="\t")